In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1569985286400_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
projects = spark.read.csv('s3://dend-capstone-lkj/projects_with_repository_fields-1.0.0-2017-06-15.csv', inferSchema=True, header=True)
versions = spark.read.csv('s3://dend-capstone-lkj/versions-1.0.0-2017-06-15.csv', inferSchema=True, header=True)
dependencies = spark.read.csv('s3://dend-capstone-lkj/dependencies-1.0.0-2017-06-15.csv', inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
num_of_projects = projects.count()
num_of_versions = versions.count()
num_of_dependencies = dependencies.count()
print(f'{num_of_projects} project records, {num_of_versions} version records, {num_of_dependencies} dependency records')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2215454 project records, 9023105 version records, 51239986 dependency records

In [5]:
projects.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Created Timestamp: string (nullable = true)
 |-- Updated Timestamp: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- Homepage URL: string (nullable = true)
 |-- Licenses: string (nullable = true)
 |-- Repository URL: string (nullable = true)
 |-- Versions Count: string (nullable = true)
 |-- SourceRank: string (nullable = true)
 |-- Latest Release Publish Timestamp: string (nullable = true)
 |-- Latest Release Number: string (nullable = true)
 |-- Package Manager ID: string (nullable = true)
 |-- Dependent Projects Count: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Last synced Timestamp: string (nullable = true)
 |-- Dependent Repositories Count: string (nullable = true)
 |-- Repository ID: string (nullable = true)
 |-- Repository Host Type: 

Only interested in fields below:

* ID
* Platform
* Name
* Dependent Repositories Count
* Repository Host Type
* Repository Name with Owner
* Repository Created Timestamp
* Repository Size
* Repository Stars Count
* Repository Language
* Repository Forks Count
* Repository Watchers Count
* Repository License
* Repository Contributors Count
* Repository Status

Let's do some selection and renaming

In [6]:
projects_selected = projects.select(
    F.col('ID').alias('project_id'),
    F.col('Platform').alias('platform'),
    F.col('Name').alias('name'),
    F.col('Dependent Repositories Count').alias('dependent_count'),
    F.col('Repository Host Type').alias('host_type'),
    F.col('Repository Name with Owner').alias('repo_name'),
    F.col('Repository Created Timestamp').alias('repo_create'),
    F.col('Repository Size').alias('size'),
    F.col('Repository Stars Count').alias('stars'),
    F.col('Repository Language').alias('language'),
    F.col('Repository Forks Count').alias('forks'),
    F.col('Repository Watchers Count').alias('watchers'),
    F.col('Repository License').alias('license'),
    F.col('Repository Contributors Count').alias('contributors'),
    F.col('Repository Status').alias('status'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
github_projs = projects_selected.where(projects_selected.host_type=='GitHub')
print(f'{github_projs.count()} projects hosted on GitHub')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1690039 projects hosted on GitHub

In [8]:
github_projs.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in github_projs.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+----+---------------+---------+---------+-----------+----+-----+--------+-----+--------+-------+------------+-------+
|project_id|platform|name|dependent_count|host_type|repo_name|repo_create|size|stars|language|forks|watchers|license|contributors| status|
+----------+--------+----+---------------+---------+---------+-----------+----+-----+--------+-----+--------+-------+------------+-------+
|         0|       0|   0|              0|        0|        0|          9| 275|   80|   42811|   12|      48| 669384|         221|1637715|
+----------+--------+----+---------------+---------+---------+-----------+----+-----+--------+-----+--------+-------+------------+-------+

It seems that there are quite a lot missing values in column "status" and "license", so these two columns can be dropped directly and null values in other columns can be cleaned.

In [9]:
# keep records with non-null values
github_projs = github_projs.drop('license', 'status').dropna(how='any')
print(f'{github_projs.count()} GitHub projects ready for analysis')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1646827 GitHub projects ready for analysis

In [10]:
versions.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Project Name: string (nullable = true)
 |-- Project ID: integer (nullable = true)
 |-- Number: string (nullable = true)
 |-- Published Timestamp: string (nullable = true)
 |-- Created Timestamp: string (nullable = true)
 |-- Updated Timestamp: string (nullable = true)

In [11]:
versions.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in versions.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+------------+----------+------+-------------------+-----------------+-----------------+
| ID|Platform|Project Name|Project ID|Number|Published Timestamp|Created Timestamp|Updated Timestamp|
+---+--------+------------+----------+------+-------------------+-----------------+-----------------+
|  0|       0|           0|         0|     0|                  0|                0|                0|
+---+--------+------------+----------+------+-------------------+-----------------+-----------------+

Great, there is no missing values in fields. Just select the interested fields and renaming.

In [12]:
versions_renamed = versions.select(
    F.col('ID').alias('version_id'),
    F.col('Platform').alias('platform'),
    F.col('Project Name').alias('name'),
    F.col('Project ID').alias('project_id'),
    F.col('Number').alias('number'),
    F.col('Published Timestamp').alias('published')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
dependencies.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Project Name: string (nullable = true)
 |-- Project ID: integer (nullable = true)
 |-- Version Number: string (nullable = true)
 |-- Version ID: integer (nullable = true)
 |-- Dependency Name: string (nullable = true)
 |-- Dependency Platform: string (nullable = true)
 |-- Dependency Kind: string (nullable = true)
 |-- Optional Dependency: string (nullable = true)
 |-- Dependency Requirements: string (nullable = true)
 |-- Dependency Project ID: string (nullable = true)

In [14]:
dependencies.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in dependencies.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+------------+----------+--------------+----------+---------------+-------------------+---------------+-------------------+-----------------------+---------------------+
| ID|Platform|Project Name|Project ID|Version Number|Version ID|Dependency Name|Dependency Platform|Dependency Kind|Optional Dependency|Dependency Requirements|Dependency Project ID|
+---+--------+------------+----------+--------------+----------+---------------+-------------------+---------------+-------------------+-----------------------+---------------------+
|  0|       0|           0|         0|             0|         0|              0|                 11|             11|           51239986|                     11|              1212888|
+---+--------+------------+----------+--------------+----------+---------------+-------------------+---------------+-------------------+-----------------------+---------------------+

We can drop "Optional Dependency" column directly and drop other missing values at all.

In [15]:
dependencies_cleaned = dependencies.drop('Optional Dependency').dropna(how='any')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
dependencies_cleaned.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in dependencies_cleaned.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+------------+----------+--------------+----------+---------------+-------------------+---------------+-----------------------+---------------------+
| ID|Platform|Project Name|Project ID|Version Number|Version ID|Dependency Name|Dependency Platform|Dependency Kind|Dependency Requirements|Dependency Project ID|
+---+--------+------------+----------+--------------+----------+---------------+-------------------+---------------+-----------------------+---------------------+
|  0|       0|           0|         0|             0|         0|              0|                  0|              0|                      0|                    0|
+---+--------+------------+----------+--------------+----------+---------------+-------------------+---------------+-----------------------+---------------------+

In [17]:
dependencies_cleaned = dependencies_cleaned.select(
    F.col('Platform').alias('platform'),
    F.col('Project Name').alias('name'),
    F.col('Version Number').alias('version_number'),
    F.col('Version ID').alias('version_id'),
    F.col('Dependency Name').alias('dependency_name'),
    F.col('Dependency Platform').alias('dependency_platform'),
    F.col('Dependency Kind').alias('dependency_kind'),
    F.col('Dependency Requirements').alias('dependency_requirements'),
    F.col('Dependency Project ID').alias('dependency_project_id'),
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# how many projects on GitHub got at least 10000 stars?
github_projs.where(github_projs.stars >= 10000).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7497

In [20]:
# export all repository names, we will use it to get the JSON records about star history 
github_projs.where(github_projs.stars >= 10000).\
    select('repo_name').write.format('text').\
    option('header', 'false').mode('append').\
    save("s3://dend-capstone-lkj/github_repos")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…